# Assemble Dataset

### 1. Imports

In [1]:
import pandas as pd
import warnings

### 2. File Settings

In [2]:
csv_playoffs = '../data/playoffs.csv'
csv_regular_season = '../data/regular_season.csv'
csv_team_lookup = '../data/team_lookup.csv'
csv_output = '../data/presidents_trophy_curse.csv'

### 3. Load Files

In [3]:
failures = 0
csv_files = [csv_playoffs, csv_regular_season, csv_team_lookup]
df_dict = dict()
for csv_file in csv_files:
    try:
        df_dict[csv_file] = pd.read_csv(csv_file)
        print('Loaded file %s' % csv_file)
    except:
        warnings.warn('Failed to load %s' % csv_file)
        failures += 1
        
if failures > 0:
    raise RuntimeError('Failed to load necessary files')

Loaded file ../data/playoffs.csv
Loaded file ../data/regular_season.csv
Loaded file ../data/team_lookup.csv


### 4. Assemble Single DataFrame

In [4]:
df = df_dict[csv_playoffs].merge(df_dict[csv_team_lookup],
                                 how='left',
                                 left_on='Team',
                                 right_on='FullName')
df = df.merge(df_dict[csv_regular_season],
              how='left',
              left_on=['Season', 'Abbreviation'],
              right_on=['Season', 'Team'])

df['SeasonRank'] = df.groupby('Season')['Rank_y'].rank('dense').astype(int)
df.drop(['Team_x', 'Team_y', 'Rank_y'], axis=1, inplace=True)
df.rename(columns={'Rank_x': 'PlayoffRank'}, inplace=True)
print(df.shape)
df.head()

(576, 7)


,PlayoffRank,GP,W,Season,FullName,Abbreviation,SeasonRank
0,1,20,15,1985,Montreal Canadiens,MTL,7
1,2,22,12,1985,Calgary Flames,CGY,6
2,3,19,10,1985,St. Louis Blues,STL,12
3,4,16,8,1985,New York Rangers,NYR,13
4,5,10,6,1985,Toronto Maple Leafs,TOR,16


### 5. Export DataFrame to CSV

In [5]:
df.to_csv(csv_output, index=False)